In [42]:
import tensorflow as tf
from keras.layers import Input,Dropout,Reshape,Permute,Conv1D,Conv3D, Permute,Flatten
from keras.models import Model,Sequential


In [43]:
model = Sequential()

In [44]:

def create_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)

    # Shape variant layers
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 2, 1), padding='same', activation='relu')(inputs)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(2, 2, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(2, 2, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(2, 2, 1), padding='same', activation='relu')(x)

    # Shape invariant layers
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu')(x)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', activation='relu')(x)

    # Flatten layer
    # Flatten layer
    x = Flatten()(x)
    
    # Reshape layer
    x = Reshape((512, 1000))(x)
    # Additional layers
    x = Permute((2, 1))(x)

    x = Conv1D(filters=3, kernel_size=1)(x)

    # Assuming you have a custom SelfAttention layer
   # x = SelfAttention(4,32)(x)

    # Additional layers
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Softmax()(x)

    # Define model
    model = Model(inputs=inputs, outputs=outputs)

    return model

# Input shape
input_shape = (32, 64, 1000, 1)  # Assuming range x doppler x time x channels

# Create the model
model = create_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 32, 64, 1000, 1   0         
                             )]                                  
                                                                 
 conv3d_262 (Conv3D)         (None, 32, 32, 1000, 32   896       
                             )                                   
                                                                 
 conv3d_263 (Conv3D)         (None, 16, 16, 1000, 32   27680     
                             )                                   
                                                                 
 conv3d_264 (Conv3D)         (None, 8, 8, 1000, 32)    27680     
                                                                 
 conv3d_265 (Conv3D)         (None, 4, 4, 1000, 32)    27680     
                                                          

In [45]:
# Define custom truncated mean squared error (T-MSE) loss
def truncated_mse(y_true, y_pred):
    # Define your truncated mean squared error calculation here
    # This is just a placeholder, replace it with your actual implementation
    return tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred)))

# Combine cross-entropy loss and truncated mean squared error (T-MSE) loss
def combined_loss(y_true, y_pred):
    cross_entropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    t_mse = truncated_mse(y_true, y_pred)
    return cross_entropy + t_mse

In [46]:
# Define metrics (accuracy in this case)
metrics = ['accuracy']

# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005),
              loss=combined_loss,
              metrics=metrics)